In [205]:
import pdfplumber
import pandas as pd

# Abre o PDF e extrai as tabelas
with pdfplumber.open("cardapio_junho.pdf") as pdf:
    page = pdf.pages[0]
    tables = page.extract_tables()
    
    # Processar o cabeçalho (tabela 0)
    # Sabemos que as posições com o texto dos dias são:
    # coluna 1, 4, 7, 10 e 13 (evitar colunas vazias)
    # e as respectivas partes na segunda linha.
    header0 = tables[0]
    day_indices = [1, 4, 7, 10, 13]
    day_names = []
    for idx in day_indices:
        # Some espaço se houver None
        part1 = header0[0][idx] or ""
        part2 = header0[1][idx] or ""
        combined = f"{part1} {part2}".strip()
        day_names.append(combined)
    
    # adiciona o identificador categoria pra primeira coluna    
    colunas_finais = ['Categoria'] + day_names
    
    
    # Processar o corpo da tabela (tabela 1)
    body_raw = tables[1]
    # Remover linhas totalmente vazias
    clean_body = [row for row in body_raw if row and any(cell and cell.strip() for cell in row)]
    
    # Devido à inconsistência, vamos ajustar cada linha.
    # Nossa estratégia:
    # - Utilizar apenas as 6 primeiras posições: a coluna 0 é a Categoria e as próximas 5 devem representar os dias.
    # - Se a célula referente à sexta-feira (índice 5) estiver vazia,
    #   mas houver conteúdo na coluna seguinte (índice 6), usamos essa informação.
    def adjust_row(row):
        # Se a linha tiver menos de 7 colunas, vamos preenchê-la com None
        row = row + [None] * (7 - len(row))
        # Se a posição 5 estiver vazia e a 6 tiver dado, substituímos:
        if (not row[5] or row[5].strip() == "") and row[6] and row[6].strip() != "":
            row[5] = row[6]
        # Retornamos apenas as primeiras 6 colunas: [Categoria, Segunda, Terça, Quarta, Quinta, Sexta]
        return row[:6]
    
    adjusted_body = [adjust_row(row) for row in clean_body if len(row) >= 6]
    
    # Criação do DataFrame com as colunas finais
    df = pd.DataFrame(adjusted_body, columns=colunas_finais)
    
    # Preenche as linhas onde a coluna Categoria está vazia com o valor anterior
    df['Categoria'] = df['Categoria'].ffill()




In [206]:
df_t = df.set_index('Categoria').T.reset_index()
df_t = df_t.rename(columns={'index': 'Dia'})

In [207]:
from collections import defaultdict

# Agrupar colunas com mesmo nome
agrupadas = defaultdict(list)

# Ignora a coluna 'Dia' (índice 0)
for col in df_t.columns[1:]:
    agrupadas[col].append(col)

# Montar novo DataFrame agrupando colunas duplicadas
df_agrupado = pd.DataFrame()
df_agrupado['Dia'] = df_t['Dia']

for categoria, col_list in agrupadas.items():
    # Para cada grupo de colunas com o mesmo nome
    if len(col_list) == 1:
        # Se só há uma coluna, usa direto
        df_agrupado[categoria] = df_t[col_list[0]]
    else:
        # Combina os valores das colunas duplicadas
        df_agrupado[categoria] = df_t[col_list].apply(
            lambda row: ' e '.join(dict.fromkeys(filter(None, row))), axis=1
        )


In [208]:
# Exibe resultado
df_agrupado

,Dia,Saladas,Acompanhamento,Guarnição,Vegetariano,Prato Principal
0,"segunda-feira, 2 de junho de 2025",Acelga com tomate picado e Beterraba cozida,Arroz Branco e Feijão Preto,Aipim souté,Omelete de forno,Iscas suínas refogadas
1,"terça-feira, 3 de junho de 2025",Chicória com cenoura\nralada e Chuchu colorido...,Arroz Branco e Feijão Carioca,Espaguete ao molho sugo,Entreveiro de legumes,Espetinho de frango
2,"quarta-feira, 4 de junho de 2025",Alface com rabanete e Grão de bico com cenoura...,Arroz Branco/arroz\ncolorido e Feijão Preto,Quirerinha temperada,Escondidinho de cabotiá\ncom proteína de soja,Bife a tomatelli
3,"quinta-feira, 5 de junho de 2025",Almeirão com cenoura e Repolho cozido\nFruta,Arroz Branco e Feijão Preto,Batata doce\ncaramelada,Mini pizza de\nabobrinha,Pernil grelhado
4,"sexta-feira, 6 de junho de 2025",Alface com repolho e Abobrinha colorida,Arroz Branco e Feijão Preto,Macarrão com ervas\nfinas,Ovo ao molho,Frango assado


In [209]:
print(df_agrupado)

                                 Dia  \
0  segunda-feira, 2 de junho de 2025   
1    terça-feira, 3 de junho de 2025   
2   quarta-feira, 4 de junho de 2025   
3   quinta-feira, 5 de junho de 2025   
4    sexta-feira, 6 de junho de 2025   

                                             Saladas  \
0        Acelga com tomate picado e Beterraba cozida   
1  Chicória com cenoura\nralada e Chuchu colorido...   
2  Alface com rabanete e Grão de bico com cenoura...   
3       Almeirão com cenoura e Repolho cozido\nFruta   
4            Alface com repolho e Abobrinha colorida   

                                Acompanhamento                  Guarnição  \
0                  Arroz Branco e Feijão Preto                Aipim souté   
1                Arroz Branco e Feijão Carioca    Espaguete ao molho sugo   
2  Arroz Branco/arroz\ncolorido e Feijão Preto       Quirerinha temperada   
3                  Arroz Branco e Feijão Preto    Batata doce\ncaramelada   
4                  Arroz Branco e Fei

In [210]:
with pdfplumber.open("cardapio_junho.pdf") as pdf:
    for i, page in enumerate(pdf.pages, start=1):
        if i == 3:
            continue
        print(f"\n📄 Página {i}")
        tables = page.extract_tables()
        if not tables:
            print("⚠️ Nenhuma tabela encontrada nesta página.")
            continue
        for t_idx, table in enumerate(tables):
            print(f"\nTabela {t_idx} da página {i}")
            for row in table:
                print(row)



📄 Página 1

Tabela 0 da página 1
['', 'segunda-feira, 2 de junho de', '', '', 'terça-feira, 3 de junho de', '', '', 'quarta-feira, 4 de junho', '', '', 'quinta-feira, 5 de', '', '', 'sexta-feira, 6 de junho', '']
[None, '2025', None, None, '2025', None, None, 'de 2025', None, None, 'junho de 2025', None, None, 'de 2025', None]

Tabela 1 da página 1
['Saladas', 'Acelga com tomate picado', 'Chicória com cenoura\nralada', 'Alface com rabanete', 'Almeirão com cenoura', '', 'Alface com repolho', '']
[None, None, None, None, None, None, '', None]
[None, 'Beterraba cozida', 'Chuchu colorido\nFruta', 'Grão de bico com cenoura\ncozida', 'Repolho cozido\nFruta', 'Abobrinha colorida', None, None]
[None, None, None, None, None, None, 'Abobrinha colorida', None]
[None, None, None, None, None, None, '', None]
[None, None, None, None, None, '', '', '']
['Acompanhamento', 'Arroz Branco', 'Arroz Branco', 'Arroz Branco/arroz\ncolorido', 'Arroz Branco', 'Arroz Branco', None, None]
[None, 'Feijão Preto',


Tabela 0 da página 2
['', 'segunda-feira, 9 de junho', '', '', 'terça-feira, 10 de junho de', '', '', 'quarta-feira, 11 de junho', '', '', 'quinta-feira, 12 de', '', '', 'sexta-feira, 13 de junho', '']
[None, 'de 2025', None, None, '2025', None, None, 'de 2025', None, None, 'junho de 2025', None, None, 'de 2025', None]

Tabela 1 da página 2
['Saladas', 'Acelga com pepino', 'Alface com repolho', 'Chicória com cebola', 'Alface com cenoura', '', 'Mix de folhas com', '']
[None, None, None, None, None, None, 'tomate', None]
[None, 'Abobrinha vinagrete', 'Beterraba cozida\nFruta', 'Vinagrete de lentilha', 'Chuchu cozido\nFruta', '', 'Mix grão', '']
[None, None, None, None, None, None, '', None]

Tabela 2 da página 2
['Acompanhamento', 'Arroz Branco', 'Arroz Branco', 'Arroz Branco', 'Arroz Branco', 'Arroz Branco']
[None, 'Feijão Preto', 'Feijão Preto', 'Feijão carioca', 'Feijão Preto', 'Feijão Carioca']

Tabela 3 da página 2
['Guarnição', 'Legumes refogados', 'Farofa colorida', 'Polenta com 